In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import mplfinance as mpf
import requests
from sklearn.metrics import mean_squared_error

In [ ]:
#API Key

key = "XTL1OOAD2EJD1V6Y"

In [ ]:
#API

base_url = "https://www.alphavantage.co/query"
function = "TIME_SERIES_DAILY"
symbol = "TSLA"
outputsize = "full"  
interval = "daily"  

In [ ]:
#API Parameters

params = {
    "function": function,
    "symbol": symbol,
    "outputsize": outputsize,
    "apikey": key,
    #"interval": interval,  
}

In [ ]:
#Calling API

response = requests.get(base_url, params=params)

In [ ]:
#Converting fetched data into json format

data = response.json()

In [ ]:
data

In [ ]:
ts_data = data["Time Series (Daily)"]

In [ ]:
#json to dataframe

df = pd.DataFrame.from_dict(ts_data, orient="index")  

In [ ]:
df.head()

In [ ]:
#renaming dataframe columns 

columns = ['open', 'high', 'low', 'close', 'volume']  

df.columns = columns 

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
#datatype conversion from object(string) to float/int(numeric)

df = df.apply(pd.to_numeric, errors='coerce')

In [ ]:
df.info()

In [ ]:
sns.set_theme(style="dark")

In [ ]:
#creating new column of "datetime" datatype

df['Date'] = pd.to_datetime(df.index)  

In [ ]:
df1 = df.sort_values('Date')

In [ ]:
df1.head()

In [ ]:
#lineplotting closing value of last 100 days

plt.figure(figsize=(18,9))
sns.lineplot(data=df1, x='Date', y='close')
plt.xlabel('')
plt.ylabel('Closing Value')
plt.title("Tesla Stock Price")
plt.tight_layout()
plt.show()

In [ ]:
df1['Date']=pd.to_datetime(df1['Date'])

In [ ]:
df1.index=df1['Date']

In [ ]:
df1 = df1.drop(columns='Date')

In [ ]:
mpf.plot(df1, type='candle', style='yahoo', mav=(5,20), figratio=(18,9))  #candlestick plotting using mpl finance lib
plt.show()

In [ ]:
# Machine Learning

In [ ]:
total_rows = len(df1)

total_rows

In [ ]:
### Seperating training_size and test_size 


In [ ]:
train_size = int(0.7*total_rows)

In [ ]:
test_size = int(0.3*total_rows)

In [ ]:
train_size

In [ ]:
test_size

In [ ]:
### Spliting data 

In [ ]:
train_data = df1[:train_size]

In [ ]:
test_data = df1[train_size:(train_size+test_size)]

In [ ]:
test_data.head()

In [ ]:
train_data.head()

In [ ]:
len(train_data)

In [ ]:
len(test_data)

In [ ]:
#from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
train_data.info()

In [ ]:
test_data.info()

In [ ]:
## Moving Average Calculation and Visualization

In [ ]:
close_price = train_data['close'].values

In [ ]:
close_price

In [ ]:
window = 20

ma = df1['close'].ewm(span=window, adjust=True).mean()

In [ ]:
ma

In [ ]:
ma.info()

In [ ]:
ind = ma.index

In [ ]:
ind

In [ ]:
fil_d = df1[df1.index.isin(ind)]

In [ ]:
fil_d

In [ ]:
avg = ma.values

In [ ]:
avg

In [ ]:
mav = pd.DataFrame(avg, columns=['price'], index=ma.index)

In [ ]:
mav

In [ ]:
plt.figure(figsize=(18,9))
sns.lineplot(data=mav, x='Date', y='price', color='blue', label='EMA')
sns.lineplot(data=fil_d, x='Date', y='close', color='red', label='Actual Closing Price')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
#normalising data 

scaler = MinMaxScaler()
scaled_train_data = scaler.fit_transform(train_data)
scaled_test_data = scaler.transform(test_data)

In [ ]:
scaled_train_data

In [ ]:
scaled_test_data

In [ ]:
x_train = []
y_train = []

In [ ]:
sequence_length = 10

In [ ]:
for i in range(len(scaled_train_data)-sequence_length):
    x_train.append(scaled_train_data[i:i+sequence_length])
    y_train.append(scaled_train_data[i+sequence_length])

In [ ]:
x_test = []
y_test = []

In [ ]:
for i in range(len(scaled_test_data)-sequence_length):
    x_test.append(scaled_test_data[i:i+sequence_length])
    y_test.append(scaled_test_data[i+sequence_length])

In [ ]:
x_train, y_train = np.array(x_train), np.array(y_train)
x_test, y_test = np.array(x_test), np.array(y_test)

In [ ]:
x_train

In [ ]:
y_train

In [ ]:
x_train.shape

In [ ]:
y_train.shape

In [ ]:
x_test.shape

In [ ]:
y_test.shape

In [ ]:
len(df1.columns)

In [ ]:
#Neural Network model

model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(100, activation='relu', input_shape=(sequence_length, 5)),
    tf.keras.layers.Dense(50),
    tf.keras.layers.Dense(5)
])

In [ ]:
#compile the model

model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
#train the model

model.fit(x_train, y_train, epochs=50, batch_size=16)

In [ ]:
test_loss = model.evaluate(x_test)

test_loss

In [ ]:
history = model.fit(x_train, y_train, epochs=50, batch_size=16, validation_split=0.3)

In [ ]:
# Plot the training loss and validation loss over epochs

plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
#predict using the created neural network

predicted_prices = model.predict(x_test)

In [ ]:
predicted_prices

In [ ]:
#retransforming the scaled data into real data

predictions_x = scaler.inverse_transform(predicted_prices)

In [ ]:
predictions_y = scaler.inverse_transform(y_test)

In [ ]:
#cost function and error

rmse = np.sqrt(mean_squared_error(y_test, predicted_prices))

In [ ]:
rmse

In [ ]:
scaled_train_data.shape

In [ ]:
predictions_x.shape

In [ ]:
predicted_prices.shape[0]

In [ ]:
train = df1[:train_size+1]
valid = df1[train_size:train_size + len(predicted_prices)]
valid.loc[:, 'Predictions'] = predictions_x[:, 3]


In [ ]:
train

In [ ]:
valid

In [ ]:
#visualising real stock value with the predicted values

plt.figure(figsize=(18,9))
plt.xlabel('Date')
plt.ylabel('Close Price')
plt.plot(valid['close'])
plt.plot(valid['Predictions'])
plt.legend([ 'Actual','Predictions'])
plt.show()